### **Exercise: Sentiment Analysis and Key Insights Extraction from Ford Car Reviews**

### **Problem Statement:**
You have been provided with a dataset containing Ford car reviews. Your task is to use LangChain and the concepts you’ve learned to perform the following tasks:

1. **Sentiment Analysis**: Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and store the result.
2. **Key Insights Extraction**: Extract key pieces of information from each review, such as the pros and cons mentioned, and the specific features the reviewer liked or disliked (e.g., vehicle performance, comfort, price).

You will build a LangChain-based solution that leverages language models to automatically extract this information and provide a structured summary of the reviews. 

---
### **Steps to Solve:**

#### **Step 1: Load the Dataset**
- The dataset file is named `ford_car_reviews.csv` and is sourced from Kaggle: [Edmunds Consumer Car Ratings and Reviews](https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews).
- For this exercise, **limit the data to the first 25 records**. This can be achieved by using `df.head(25)` or `df.iloc[:25]` when loading the data into a DataFrame.

#### **Step 2: Define the Sentiment Analysis Task**
- Use LangChain to create a pipeline to classify the sentiment of each review.
- Define prompts that can guide the model to evaluate the sentiment. For example:
  - "Given the following car review, classify the sentiment as positive, neutral, or negative."

#### **Step 3: Key Insights Extraction**
- Use LangChain to create a pipeline to extract pros, cons, and notable features from each review. Define prompts such as:
  - "What are the pros and cons of the vehicle described in the following review?"
  - "What specific features of the vehicle does the reviewer like or dislike?"

#### **Step 4: Update the DataFrame with New Information**
- Run the pipeline for each review and collect the sentiment and insights.
- Once the analysis and extraction are complete, update the original DataFrame with additional columns to include:
  - Sentiment (positive, neutral, negative)
  - Pros
  - Cons
  - Liked_Features
  - Disliked_Features

---

### **Example Output:**

```json
{
  "Review_Date": "03/07/13",
  "Vehicle_Title": "2006 Ford Mustang Coupe",
  "Review_Text": "With the expected arrival of our 6th child...",
  "Rating": 4.125,
  "Sentiment": "Positive",
  "Pros": "Good driving experience, Large seating capacity, Great options",
  "Cons": "None mentioned",
  "Liked_Features": ["Driving experience", "Seating capacity", "Options available"],
  "Disliked_Features": []
}
```

In [52]:
import pandas as pd
df=pd.read_csv('ford_car_reviews.csv',engine='python')

In [53]:
df.drop("Unnamed: 0", axis=1,inplace=True)
df=df.iloc[:25]

In [54]:
df.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0
1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0
2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0
3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0
4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0


In [68]:
import os, json, re, getpass
from dotenv import load_dotenv

load_dotenv(".env", override=True)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key:")

In [69]:
# LLM
from langchain_groq import ChatGroq

model_id = "llama3-8b-8192" #llama3-8b-8192, llama-3.1-8b-instant, llama3-groq-8b-8192-tool-use-preview, llama3-groq-70b-8192-tool-use-preview
llm = ChatGroq(model_name=model_id, temperature=0, )

In [70]:
from langchain_core.prompts import ChatPromptTemplate
combined_system_message_template = """
Given the following car review, perform the following tasks:
1. Classify the sentiment as Positive, Neutral, or Negative.
2. Identify the following details:
   - Pros: List the positive aspects of the vehicle mentioned in the review.
   - Cons: List the negative aspects of the vehicle mentioned in the review.
   - Liked Features: Highlight specific features the reviewer liked (if mentioned).
   - Disliked Features: Highlight specific features the reviewer disliked (if mentioned).
Respond with a JSON object containing these fields: Sentiment, Pros, Cons, Liked_Features, Disliked_Features.
"""
combined_template = ChatPromptTemplate([
    ("system", combined_system_message_template),
    ("human", "{user_input}"),
])

In [78]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

def analyze_review(review):
    prompt = combined_template.format(user_input=review)
    response = llm.invoke(prompt)
    json_response = JsonOutputParser().invoke(response)
    try:
        result = json_response
    except json.JSONDecodeError:
        result = {
            "Sentiment": "Neutral",  # Default to Neutral if parsing fails
            "Pros": "Could not extract",
            "Cons": "Could not extract",
            "Liked_Features": [],
            "Disliked_Features": []
        }
    return result

# Process the dataset and generate JSON output
output_list = []

In [79]:
for index, row in df.iterrows():
    review_data = {
        "Review_Date": row['Review_Date'],  # Replace with actual column name
        "Vehicle_Title": row['Vehicle_Title'],  # Replace with actual column name
        "Review_Text": row['Review'],  # Replace with actual column name
        "Rating": row['Rating'],  # Replace with actual column name
    }
    # Analyze the review for sentiment and features
    analysis = analyze_review(review_data["Review_Text"])
    review_data.update(analysis)
    output_list.append(review_data)

In [83]:
for i in output_list:
    print(i,"\n\n")

{'Review_Date': ' on 06/06/18 14:19 PM (PDT)', 'Vehicle_Title': '2006 Ford Mustang Coupe GT Premium 2dr Coupe (4.6L 8cyl 5M)', 'Review_Text': ' Doesn’t disappoint', 'Rating': 5.0, 'Sentiment': 'Positive', 'Pros': [], 'Cons': [], 'Liked_Features': [], 'Disliked_Features': []} 


{'Review_Date': ' on 08/12/17 06:06 AM (PDT)', 'Vehicle_Title': '2006 Ford Mustang Coupe V6 Standard 2dr Coupe (4.0L 6cyl 5M)', 'Review_Text': ' I bought mine 4/17 with 98K. Have been wanting a V6 5-sp, \'05-\'09 vintage for years. The engine is fine. Sounds good. Great mileage. Good power. I pride myself on smooth take-off and gear changes, but this is the orneriest transmission I\'ve ever used! The difference between idle and 4000 rpm is about 1/8" on the gas pedal, so starting-out without either stalling or way over-revving takes a LOT of finesse. Gear changes are very difficult to master smoothly without lurching. The ride is very harsh with a lot of road noise, which I suppose goes with a quasi high-perform